In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

/home/kenn/anaconda3/envs/tf_base/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
def add_layer(inputs, in_size,out_size, n_layer,keep_prob, activation_function=None):
    layer_name = 'fc_%s' % n_layer
    # 把所有内容可视化的时候包含在layer节点内
    with tf.name_scope(layer_name):
        
        with tf.name_scope("weights"):
            Weights = tf.Variable(tf.random_normal([in_size, out_size],stddev=0.1), name="W") # 权重矩阵 行=输入维度，列=输出维度（下一层输入维度）
            # 把参数数据总结到直方图中
            tf.summary.histogram(layer_name+"/weights", Weights)
            
        with tf.name_scope("biases"):
            #biases = tf.Variable(tf.zeros([1,out_size]) + 0.1, name="b")           # 偏置向量 行=1，列=输出维度
            biases = tf.Variable(tf.zeros([out_size]) + 0.1, name="b") 
            tf.summary.histogram(layer_name+"/biases", biases)
            
        with tf.name_scope("Wx_plus_b"):
            Wx_plus_b = tf.matmul(inputs, Weights) + biases              # 其实是xW，如果行列定义反过来，这里可以Wx，xW的比较符合数据储存直觉
            
        if activation_function is None:
            outputs = Wx_plus_b
        else:
            outputs = activation_function(Wx_plus_b)
            
        # 加入dropout，rate由keep_prob决定
        outputs = tf.nn.dropout(outputs, keep_prob)
        tf.summary.histogram(layer_name+"/outputs", outputs)
        return outputs


    
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, strides, padding):
    # strides格式:1,x,y,1，开头结尾是1，中间2个是x和y方向的步长
    return tf.nn.conv2d(x, W, strides=strides, padding=padding)

def max_pool(x,ksize,strides,padding):
    return tf.nn.max_pool(x, ksize=ksize,strides=strides,padding=padding)
    # ksize格式:1,x,y,1，开头结尾是1，中间2个是x和y方向的步长

def add_conv_layer(x_image , n_layer, kernel_size=[5,5,1,32], activation_function=tf.nn.relu,strides=[1,1,1,1], padding="SAME"):
    layer_name = 'conv_%s' % n_layer
    with tf.name_scope(layer_name):
        
        with tf.name_scope("weights"):
            W_conv = weight_variable(kernel_size) 
            tf.summary.histogram(layer_name+"/weights", W_conv)
            
        with tf.name_scope("biases"):
            b_conv = bias_variable([kernel_size[3]]) 
            tf.summary.histogram(layer_name+"/biases", b_conv)
            
        with tf.name_scope("h_conv"):
            h_conv = activation_function(conv2d(x_image, W_conv,strides=strides,padding=padding) + b_conv) #因为是samepadding，步长1，得到的结果是28x28x32\
            tf.summary.histogram(layer_name+"/conv", h_conv)
        return h_conv
    
def add_pool_layer(h_conv,n_layer,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME"):
    layer_name = 'pool_%s' % n_layer
    with tf.name_scope(layer_name):
        with tf.name_scope("h_pool"):
            h_pool = max_pool(h_conv,ksize=ksize,strides=strides,padding=padding) 
            tf.summary.histogram(layer_name+"/pool", h_pool)
        return h_pool
    

    
    
def compute_accuracy(v_xs, v_ys):
    global prediction #先把prediction定义为全局变量
    y_pre = sess.run(prediction, feed_dict={xs:v_xs,keep_prob_l1:1, keep_prob_l2:1}) #生成预测值（概率），10分类，所以一个样本是10列概率
    correct_prediction = tf.equal(tf.argmax(y_pre,1), tf.argmax(v_ys,1)) #比较概率最大值的位置和真实标签位置是否一样，一样就是true
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32)) #计算平均正确率
    result = sess.run(accuracy, feed_dict={xs:v_xs, ys:v_ys}) #生成正确率
    return result



# 只要定义为placeholder的都需要feed传入
with tf.name_scope("dropout_rate_1"):
    keep_prob_l1 = tf.placeholder(tf.float32)
with tf.name_scope("dropout_rate_2"):
    keep_prob_l2 = tf.placeholder(tf.float32)
    
    
with tf.name_scope("inputs"):
    xs = tf.placeholder(tf.float32, [None,784], name = "x_input") # 输入数据：None不限制样本数，784=28x28,每个样本784个像素点（特征）
    ys = tf.placeholder(tf.float32, [None,10], name = "y_input") # 输出数据：None不限制样本数，10个输出（10分类问题）
    # cnn中需要将输入数据reshape为图片形式
    x_image = tf.reshape(xs,[-1,28,28,1]) # -1:不考虑样本数，28,28:784像素=28x28,1：一个通道，黑白图片
    #print(x_image.shape)

    

# c1:6个5x5的卷积核,SAME padding，得到边长 = 28x28x6
conv1 = add_conv_layer(x_image,"1",[5,5,1,6],padding="SAME",activation_function=tf.nn.relu)
# s2:pool核2x2，步长2，28x28x6变成14x14x6
pool2 = add_pool_layer(conv1,"2")

# c3:16个5x5的卷积核，得到变成14-5+1 = 10x10x16
conv3 = add_conv_layer(pool2,"3",[5,5,6,16],padding="VALID",activation_function=tf.nn.relu)
# s4:pool核2x2，步长2，10x10x16变成5x5x16
pool4 = add_pool_layer(conv3,"4")

# c5:120个5x5卷积核，得到变成5-5+1 = 1x1x120
conv5 = add_conv_layer(pool4,"5",[5,5,16,120],padding="VALID",activation_function=tf.nn.relu)

##flat
flat = tf.reshape(conv5,[-1,1*1*120])

fc6 = add_layer(flat, 1*1*120, 84, n_layer="6", keep_prob = keep_prob_l1 ,activation_function=tf.nn.relu)

prediction = add_layer(fc6, 84, 10, n_layer="prediction", keep_prob = 1,activation_function=tf.nn.softmax)

    
with tf.name_scope("corss_entropy"):
    corss_entropy =  tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
    tf.summary.scalar("corss_entropy",corss_entropy)
    
with tf.name_scope("train"):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(corss_entropy)

sess = tf.Session()

# 将上述可视化合并
merged = tf.summary.merge_all()

# 将以上结构写入文件，分为train和test
train_writer = tf.summary.FileWriter("logs/LeNet_5/train",sess.graph)
test_writer = tf.summary.FileWriter("logs/LeNet_5/test",sess.graph)
# 终端激活对应环境后执行 tensorboard --logdi'file:///home/kenn/tensorflow_learning/logs'
# localhost:6006看

init = tf.global_variables_initializer()
sess.run(init)
for i in range(10000):
    batch_xs,batch_ys = mnist.train.next_batch(100)
    #通过placeholder定义输入的话，都要用feed_dict载入数据，dropout参数也要从这里输入
    sess.run(train_step, feed_dict = {xs:batch_xs, ys:batch_ys, keep_prob_l1:0.5, keep_prob_l2:0.3})
    
    if i%500 == 0:
        print(compute_accuracy(mnist.test.images, mnist.test.labels))
        
        #预测的时候droprate为0
        train_result = sess.run(merged, feed_dict = {xs:batch_xs, ys:batch_ys, keep_prob_l1:1, keep_prob_l2:1})
        test_result = sess.run(merged, feed_dict = {xs:mnist.test.images, ys:mnist.test.labels, keep_prob_l1:1, keep_prob_l2:1})
        
        train_writer.add_summary(train_result,i)
        test_writer.add_summary(test_result,i)

0.0994
0.8842
0.9266
0.9454
0.953
0.9604
0.9632
0.9689
0.97
0.9731
0.9747
0.9761
0.9774
0.9785
0.9803
0.9801
0.9815
0.982
0.9828
0.9828
